## Hashable-объекты

In [157]:
class User:
    def __init__(self, name: str, surname: str, is_active: bool = False):
        self.name = name
        self.surname = surname
        self.is_active = is_active


In [158]:
u = User(name='User', surname='Petrov')
print(u.name)
u.name = 'Petr'
print(u.name)

User
Petr


**Ответьте на вопросы**: 
  - можно ли использовать объекты класса User в качестве ключа в словаре?
  - являются ли объекты класса User хэшируемыми?
  - являются ли объекты класса User изменяемыми?

In [15]:
data = (1, 2, 3)
hash(data)

529344067295497451

In [16]:
hash([1, 2, 3])

TypeError: unhashable type: 'list'

#### Являются ли объекты класса User hashable?

In [160]:

petr = User(name='Petr', surname='Petrov')
petr_clone = User(name='Petr', surname='Petrov')

user_age_map = {
    petr: 20, 
    petr_clone: 26,
}
print(user_age_map)

print(hash(petr_clone))
print(hash(petr))


{<__main__.User object at 0x1185d15e0>: 20, <__main__.User object at 0x1185d1070>: 26}
293982471
293982558


In [161]:
user_age_map[petr]

20

Видим, что объекты являются хэшируемыми, но есть нюанс..

In [162]:
petr2 = User(name='Petr', surname='Petrov')
user_age_map[petr2]

KeyError: <__main__.User object at 0x11924aff0>

In [164]:
user_age_map1 = {
    'Petr Petrov': 20
}
user_age_map1['Petr Petrov']


20

Ожидаемое поведение аналогично строкам, например:

In [26]:
user_age_map1 = {
    'Petr Petrov': 20,
    'Petr Petrov': 26
}

In [28]:
user_age_map1


{'Petr Petrov': 26}

In [30]:
hash('Petr Petrov')

8397874054901821506

In [32]:
hash('Petr Petrov')

8397874054901821506

In [165]:
petr == petr_clone

False

Т.е. hash-значения объектов равны

In [166]:
'Petr Petrov' == 'Petr Petrov'

True

А вот hash-значения объектов класса User отличаются

In [37]:
User(name='Petr', surname='Petrov') == User(name='Petr', surname='Petrov')

False

#### Как реализовать hashable-объекты правильно? Модель данных!

In [109]:
class User:
    def __init__(self, name: str, surname: str, is_active: bool = False):
        self.__name = name
        self.__surname = surname
        self.__is_active = is_active

    def __hash__(self):
        return hash((self.__name, self.__surname))

    def __eq__(self, other):
        return self.__name == other.__name and self.__surname == other.__surname


In [110]:

petr = User(name='Petr', surname='Petrov')
petr_clone = User(name='Petr', surname='Petrov')

user_age_map = {
    petr: 20, 
    petr_clone: 26,
}
print(user_age_map)
user_age_map[User(name='Petr', surname='Petrov')]

{<__main__.User object at 0x119339130>: 26}


26

In [78]:
print(f'{hash(petr)} =? {hash(petr_clone)}')

5870677628674637673 =? 5870677628674637673


In [64]:
hash(('Petr', 'Petrov'))

5870677628674637673

In [72]:
hash(petr) == hash(petr_clone) == hash(User(name='Petr', surname='Petrov'))

True

In [73]:
petr == petr_clone == User(name='Petr', surname='Petrov')

False

In [98]:
bucket = [
    (('Petr', 'Ivanov'), 22), 
    (('Petr', 'Petrov'), 26), 
    (('Petr', 'Abramov'), 75)
]
key = ('Petr', 'Petrov')

for value, age in bucket:
    if key == value:
        print(age)

26


In [99]:
bucket = [
    (petr, 22), 
    (petr_clone, 26), 
]
key = User(name='Petr', surname='Petrov')

for value, age in bucket:
    if key == value:
        print(age)

22
26


In [114]:
petr = User(name='Petr', surname='Petrov')
ivan = User(name='Ivan', surname='Ivanov')
print(f'hash before = {hash(petr)}')
user_age_map = {
    petr: 20, 
    ivan: 26,
}

petr.name = 'Fedr'
print(f'hash after = {hash(petr)}')

print(user_age_map)
user_age_map[petr]

hash before = 5870677628674637673
hash after = 5870677628674637673
{<__main__.User object at 0x119339370>: 20, <__main__.User object at 0x11933b5f0>: 26}


20

#### Еще пример

In [124]:
class Point:
    def __init__(self, x: int, y: int):
        self.__x = x
        self.__y = y
    
    def __eq__(self, other):
        return self.__x == other.__x and self.__y == other.__y
    
    def __hash__(self):
        return hash((self.__x, self.__y))
    
    def __repr__(self):  # Point(x=1, y=2)
        return f'Point(x={self.__x}, y={self.__y})'

    def __str__(self):  # (1, 2)
        return f'({self.__x}, {self.__y})'
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y


point_a = Point(1, 1)
point_b = Point(1, 1)
print(point_a.x)
points = {point_a, point_b, Point(0, 0), point_a}
print(points)

print(hash(point_a))
print(hash(point_b))

print(point_a)
print(repr(point_a))


1
{Point(x=1, y=1), Point(x=0, y=0)}
8389048192121911274
8389048192121911274
(1, 1)
Point(x=1, y=1)


#### Есть ли способ проще? Dataclass

In [129]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Point:
    x: int
    y: int


point_a = Point(1, 1)
point_b = Point(1, 1)
{point_a, point_b, Point(0, 0), point_a}

{Point(x=0, y=0), Point(x=1, y=1)}

In [26]:
point_a == point_b

False

In [37]:
hash((10, 1)) % 20

4

In [10]:
{(1, 1), (1, 1), (0, 0)}

{(0, 0), (1, 1)}

In [127]:
user_age_map = {
    'petr': 20, 
    'petr': 26,
}
print(user_age_map)

{'petr': 26}


In [131]:
@dataclass(slots=True)
class Point:
    x: int
    y: int


In [146]:
point_a = Point(1, 1)
point_a.x = 2
print(point_a)

In [138]:
class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

point_a = Point(1, 1)
point_a.z = 3
point_a.__dict__

{'x': 1, 'y': 1, 'z': 3}

In [141]:
class Point:
    __slots__ = ['x', 'y']
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y


In [144]:
point_a = Point(1, 1)
point_a.z = 3

AttributeError: 'Point' object has no attribute 'z'

### P.S. Модель данных в Python
https://docs.python.org/3/reference/datamodel.html

In [155]:
class FriendCollection:
    def __init__(self, my_friends: list[str], wife_friends: list[str]):
        self._my_friends = my_friends
        self._wife_friends = wife_friends
    
    @property
    def _friends(self):
        return self._my_friends + self._wife_friends

    def __str__(self):
        return ', '.join(self._friends)
    
    def __len__(self):
        return len(self._friends)

    def __iter__(self):
        return iter(self._friends)

    # def __setattr__(self):
    #     raise NotImplemented()

friends = FriendCollection(['Петя', 'Вася'], ['Катя', 'Наташа'])
print(f'Friends count: {len(friends)}')
print(f'All friends: {friends}')

for friend in friends:
    print(friend)

Friends count: 4
All friends: Петя, Вася, Катя, Наташа
Петя
Вася
Катя
Наташа
